# Cargo el CSV de reproducciones eventos (Reproducciones y "No Reproducciones")

In [ ]:
# Conecta con gdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importa el CSV
import pandas as pd, numpy as np
dataframe = pd.read_csv('drive/MyDrive/AI/entrega_obligatorio/eventos_20201201_20201203_mas_10_noreps.csv')
# Elimina filas con columnas con valores Nan 
for column in list(dataframe.columns):
  nan_rows = dataframe[dataframe[column].isnull()]

In [ ]:
# Muestra Dataframe
dataframe

,usuario,id_pais,id_aplicacion,id_track,id_album,id_artista,id_playlist,id_genero,fecha_lanzamiento,reproduccion
0,12462561083,52,6,101269,9569,949,51322,32,1230768000,0
1,12462561083,52,6,10172245,1876931,1043198,51322,4,1555632000,0
2,12462561083,52,6,10260957,1878464,838972,33022,1041,1555286400,0
3,12462561083,52,6,10478308,1931685,1076154,51322,929,1557446400,0
4,12462561083,52,6,10573965,1948449,1086372,14117,868,1558224000,0
...,...,...,...,...,...,...,...,...,...,...
5513957,59996980044,531,6,33439640,6370070,120066,91959,21,1606694400,1
5513958,18763095386,388,6,33444360,6371088,379828,-1,21,1606780800,1
5513959,18768629026,388,6,33444360,6371088,379828,-1,21,1606780800,1
5513960,50373702509,222,6,33453744,6372303,363687,-1,4,1606780800,1


#Normalización
El dataset contiene columnas con identificadores, donde alguna de ellas tienen números muy altos (varios millones) , por este motivo es necesario la normalización de las columnas, para evitar la sobrerepresentación de alguna de ellas.

Se guardan parámetros de normalización (variable meanStd) para utilizar posteriormente al invocar el modelo.


In [ ]:
# Castea a double y normalizo todas las columnas
columns = list(dataframe.columns)
meanStd = list()
 
NormDataframe = dataframe
 
for column in columns:
    mean =  dataframe[column].astype(np.double).mean()
    std = dataframe[column].astype(np.double).std()
    max = dataframe[column].astype(np.double).max()
    
    if column == 'fecha_lanzamiento':
      if len(dataframe[dataframe[column].isnull()]) == 0:
        NormDataframe[column] = (mean / std)
      else:
        NormDataframe[column] = (dataframe[column].astype(np.double) - mean) / std
      meanStd.append([-1, -1, -1])
    elif column == 'reproduccion': 
      1==1
    else:
      meanStd.append([mean, std, -1])
      NormDataframe[column] = (dataframe[column].astype(np.double) - mean) / std

In [ ]:
# Muestra los datos de promedio y desviacion standard de cada columna (variable meanStd)
meanStd

[[206908860914.6384, 251587825784.14664, -1],
 [348.20748510780453, 210.2791382386964, -1],
 [6.011691049013395, 2.2632218262099517, -1],
 [11670313.054696424, 10838318.443102755, -1],
 [2229712.8013169114, 2121160.5107313045, -1],
 [432378.42190751404, 697693.858503115, -1],
 [30650.722141538154, 44067.83528885183, -1],
 [369.7700869175377, 491.60563500878794, -1],
 [-1, -1, -1]]

In [ ]:
# Muestra Dataframe Normalizado
NormDataframe

,usuario,id_pais,id_aplicacion,id_track,id_album,id_artista,id_playlist,id_genero,fecha_lanzamiento,reproduccion
0,-0.772876,-1.408639,-0.005166,-1.067421,-1.046665,-0.618365,0.469079,-0.687075,3.058823,0
1,-0.772876,-1.408639,-0.005166,-0.138220,-0.166315,0.875484,0.469079,-0.744032,3.058823,0
2,-0.772876,-1.408639,-0.005166,-0.130035,-0.165593,0.582768,0.053810,1.365383,3.058823,0
3,-0.772876,-1.408639,-0.005166,-0.109981,-0.140502,0.922719,0.469079,1.137558,3.058823,0
4,-0.772876,-1.408639,-0.005166,-0.101155,-0.132599,0.937365,-0.375188,1.013475,3.058823,0
...,...,...,...,...,...,...,...,...,...,...
5513957,-0.583939,0.869285,-0.005166,2.008552,1.951930,-0.447635,1.391225,-0.709451,3.058823,1
5513958,-0.747833,0.189237,-0.005166,2.008988,1.952410,-0.075320,-0.695558,-0.709451,3.058823,1
5513959,-0.747811,0.189237,-0.005166,2.008988,1.952410,-0.075320,-0.695558,-0.709451,3.058823,1
5513960,-0.622189,-0.600190,-0.005166,2.009853,1.952983,-0.098455,-0.695558,-0.744032,3.058823,1


#Separamos el dataset en conjunto de test y de validación

In [ ]:
# Porcentaje de datos para training.
p_train = 0.70 

# Define las columnas que seran de para training y validacion agregando una columna
NormDataframe = NormDataframe.sample(frac=1)
NormDataframe['is_train'] = np.random.uniform(0, 1, len(NormDataframe)) <= p_train
trainX, testX = NormDataframe[NormDataframe['is_train']==True], NormDataframe[NormDataframe['is_train']==False]

# Extrae de ambos dataframes los valores de evaluacion de la columna reproduccion
y_train_dataframe = trainX['reproduccion']
y_test_dataframe = testX['reproduccion']

# Elimina de ambos dataframes la columna reproduccion
trainX = trainX.drop(['reproduccion'], axis=1)
testX = testX.drop(['reproduccion'], axis=1)

# Elimina de ambos dataframes la columna is_train utilizada para la partición del dataset
testX = testX.drop(['is_train'], axis=1)
trainX = trainX.drop(['is_train'], axis=1)

In [ ]:
# Convierte los dataframes de test y validacion a Numpy Arrays
X = trainX.to_numpy()
X_valid = testX.to_numpy()

# Convierte a Numpy Arrays y transpone los vectores de resultados
y_aux = y_train_dataframe.to_numpy();
y = np.array( list(map(lambda x: 0 if x else 1, y_aux)) )[np.newaxis]
y = y.T
y_valid_aux = y_test_dataframe.to_numpy()
y_valid = np.array( list(map(lambda x: 0 if x else 1, y_valid_aux)) )[np.newaxis]
y_valid = y_valid.T

# Genera los Dataloaders
Crea Dataloaders de Pytorch que facilitan el procesamiento en batch y proveen de segmentos de datos "shufleados" para evitar sesgos en el ajuste de parámetros

In [ ]:
import torch
import torch.optim as optim # Contains optimizers such as SGD, which update the weights of Parameter during the backward step
import torch.nn as nn
import torch.nn.functional as F # contains activation functions, loss functions, etc, as well as non-stateful versions of layers such as convolutional and linear layers
from torch.utils.data import TensorDataset, DataLoader

# Convierte los Numpy Arrays a Tensores de Pytorch de tipo DoubleTensor
X, y, X_valid, y_valid = map(torch.DoubleTensor, (X, y, X_valid, y_valid))

# Crea los Dataloaders de entrenamiento y validacion
train_ds = TensorDataset(X, y)
train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
valid_ds = TensorDataset(X_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=256, shuffle=True)

#Funciones para calculo de Metricas

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
import pandas as pd, numpy as np

# Define funciones con Metricas para evaluación durante ejecución
def metrics(umbral, pred):

  numpy_valid = y_valid.detach().numpy()
  numpy_pred = pred.detach().numpy()
  numpy_pred=numpy_pred[:,-1]
  
  cant_valid = 0
  cant_pred = 0
  falso_pos = 0
  for a, b in zip(numpy_valid, numpy_pred):    
    if a == 1:
      cant_valid = cant_valid+1
    if b >= umbral:
      cant_pred = cant_pred+1
    if a == 0 and b >= umbral:
      falso_pos = falso_pos+1

  numpy_pred_bin = np.array( list(map(lambda x: 0 if x < umbral else 1, numpy_pred)) )
  print(f"real {cant_valid} pred: {cant_pred} fp: {falso_pos} tf: {cant_pred-falso_pos} precision {precision_score(numpy_valid, numpy_pred_bin, average='macro')}")

def metrics2(umbral, pred, y_valid):
  numpy_valid = y_valid.detach().numpy()
  numpy_pred = pred.detach().numpy()  
  cant_valid = 0
  cant_negativo = 0
  cant_predT = 0
  cant_predF = 0
  falso_pos = 0
  falso_negativo = 0
  true_negativo = 0
  for a, b in zip(numpy_valid, numpy_pred):    
    if a == 1:
      cant_valid += 1
    else:
      cant_negativo += 1
   
    if b >= umbral:
      cant_predT += 1      
      if a == 0:
        falso_pos += 1     
    else:
      cant_predF += 1          
      if a==0:
        true_negativo += 1     
      if a==1:
        falso_negativo += 1             
  
  numpy_pred_bin = np.array( list(map(lambda x: 0 if x < umbral else 1, numpy_pred)) )
  validPredT =(cant_predT-falso_pos)* 100 / (cant_valid)
  validPredF =(cant_predF-falso_negativo)* 100 / (cant_negativo)
  print(f"umbral:{umbral} pos:{cant_valid}  pred:{cant_predT} fp:{falso_pos} fn:{falso_negativo} tp:{cant_predT-falso_pos} neg:{cant_negativo} tn:{true_negativo} precision:{precision_score(numpy_valid, numpy_pred_bin, average='macro')} F1 :{f1_score(numpy_valid, numpy_pred_bin)} predicciones acertadas POS:{validPredT} % NEG:{validPredF} %")
  #sklearn.metrics.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')

In [ ]:
# Guarda la hora
import time;
ts = time.time()

In [ ]:
# Chequea que este habilitado el entrenamiento por GPU
print(torch.cuda.is_available())

False


# Define el Modelo

In [ ]:
from datetime import datetime
 
date_time_running = datetime.now().strftime("%m%d%Y%H%M%S")
 
# Define tamaño del input y maximo de neuronas en las capas centrales
input_size = X.shape[1]
hidden_size = 72
 
# Define el modelo
model = nn.Sequential(
    
          nn.Linear(input_size, int(hidden_size/8)),
          nn.LeakyReLU(),
          nn.Linear(int(hidden_size/8), int(hidden_size/4)),
          nn.LeakyReLU(),
          nn.Linear(int(hidden_size/4), int(hidden_size/2)),
          nn.LeakyReLU(),
          nn.Linear(int(hidden_size/2), hidden_size),
          nn.LeakyReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.LeakyReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.LeakyReLU(),    
          nn.Linear(hidden_size, hidden_size),
          nn.LeakyReLU(), 
          nn.Linear(hidden_size, hidden_size),
          nn.LeakyReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.LeakyReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.LeakyReLU(),
          nn.Linear(hidden_size, int(hidden_size/2)),
          nn.LeakyReLU(),
          nn.Linear(int(hidden_size/2), int(hidden_size/4)),
          nn.LeakyReLU(),
          nn.Linear(int(hidden_size/4), int(hidden_size/8)),
          nn.LeakyReLU(),
          nn.Linear(int(hidden_size/8), 1),
          nn.Sigmoid()
        )
 
# Castea los tensores a FloatTensor
X = X.type(torch.FloatTensor)
X_valid = X_valid.type(torch.FloatTensor)
 
y = y.type(torch.FloatTensor)
y_valid = y_valid.type(torch.FloatTensor)
 
# Crea los DataLoaders
train_ds = TensorDataset(X, y)
train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)
 
valid_ds = TensorDataset(X_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=256, shuffle=True)
 
# Define Epochs y Learning Rate
alpha = 0.001
epochs = 200

#loss = nn.BCELoss()
loss = nn.MSELoss()

opt = optim.Adam(model.parameters(), lr=alpha)
 
 
torch.save(train_dl, f"drive/MyDrive/AI/entrega_obligatorio/modelos/train_dl_{date_time_running}.DS")
torch.save(valid_dl, f"drive/MyDrive/AI/entrega_obligatorio/modelos/valid_dl_{date_time_running}.DS")
torch.save(meanStd, f"drive/MyDrive/AI/entrega_obligatorio/modelos/meanStd_{date_time_running}.DS")


dif = 0
minLoss= 10000
epocMinLoss = 0
for epoch in range(epochs):

    if epoch%5 == 1:
      for g in opt.param_groups:
        g['lr'] = g['lr']*0.9

    model.train()
    for Xb, yb in train_dl:
        # Fwd
        pred = model(Xb)
        output = loss(pred, yb)
        # Bwd
        output.backward()
        # Update params
        opt.step()
        opt.zero_grad()
 
    model.eval()
    with torch.no_grad():
        valid_loss = sum(loss(model(Xb), yb) for Xb, yb in valid_dl)
        
        if valid_loss < minLoss:
          minLoss=valid_loss
          epocMinLoss=epoch
          torch.save(model, f"drive/MyDrive/AI/entrega_obligatorio/modelos/model-MSELoss-Adam-alfa-{alpha}-{date_time_running}-{minLoss}.mod")
          #torch.save(model, f"drive/MyDrive/AI/modelos/model-MSELoss-Adam-alfa-{alpha}-{date_time_running}-{minLoss}.mod")
          pred = model(X_valid)          
          metrics2(0.5, pred, y_valid)
        print(f"epoc: {epoch} loss {valid_loss} dif paso anterior {valid_loss-dif} minLoss {minLoss} epocMinLoss {epocMinLoss} time {datetime.now()} " )
        dif = valid_loss
        if 10 > dif:
            break


#Métricas

In [ ]:
X_valid= valid_dl.dataset.tensors[0]
y_valid= valid_dl.dataset.tensors[1]
 
pred = model(X_valid)
print(pred.shape)
 
metrics2(0.5, pred, y_valid)
metrics2(0.55, pred, y_valid)
metrics2(0.6, pred, y_valid)
metrics2(0.65, pred, y_valid)
metrics2(0.7, pred, y_valid)
metrics2(0.75, pred, y_valid)
metrics2(0.8, pred, y_valid)
metrics2(0.85, pred, y_valid)
metrics2(0.9, pred, y_valid)

Evaluación de caso 

In [ ]:
def normalize(dataframe, meanStd):
  NormDataframe = dataframe
  columns = list(dataframe.columns)

  for column, m in zip(columns, meanStd):
    print(column)
    if column == 'fecha_reproduccion':
      NormDataframe[column] = dataframe[column].astype(np.double) / m[2]
    elif column == 'reproduccion': 
      1==1
    else:
      NormDataframe[column] = (dataframe[column].astype(np.double) - m[0]) / m[1]
   
  return NormDataframe


import pandas as pd, numpy as np
dataframe_part = pd.read_csv('drive/MyDrive/AI/evento_particular.csv')
dataframe_part = normalize(dataframe_part, meanStd)

dataframe_part = dataframe_part.drop(['reproduccion'], axis=1)

X_part = torch.from_numpy(dataframe_part.to_numpy())
X_part = X.type(torch.FloatTensor)
#print(X_part)

pred = model(X_part)

pred.item()

In [ ]:
from sklearn.metrics import precision_score

#torch.save(model,'drive/MyDrive/AI/model50relusgd72020.mod')
cant_valid = 0
cant_pred = 0
falso_pos = 0

umbral = 0.7
pred = model(X_valid)

numpy_valid = y_valid.detach().numpy()
numpy_pred = pred.detach().numpy()
numpy_pred=numpy_pred[:,-1]


#print(numpy_pred[:,-1])
for a, b in zip(numpy_valid, numpy_pred):    
  if a == 1:
    cant_valid = cant_valid+1
  if b >= umbral:
    cant_pred = cant_pred+1
  if a == 0 and b >= umbral:
    falso_pos = falso_pos+1

print(cant_valid)
print(cant_pred)
print(falso_pos)
print(cant_pred-falso_pos)

numpy_pred_bin = np.array( list(map(lambda x: 0 if x < umbral else 1, numpy_pred)) )
res = precision_score(numpy_valid, numpy_pred_bin, average='macro')
print(res)

 

# Ejecuto Pruebas Particulares

In [ ]:

def normalize(dataframe, meanStd):
  NormDataframe = dataframe
  columns = list(dataframe.columns)

  for column, m in zip(columns, meanStd):
    print(column)
    if column == 'dia_semana':
      NormDataframe[column] = dataframe[column].astype(np.double) / 7
    elif column == 'fecha_reproduccion':
      NormDataframe[column] = dataframe[column].astype(np.double) / m[2]
    elif column == 'fecha_lanzamiento':
      if len(dataframe[dataframe[column].isnull()]) == 0:
        NormDataframe[column] = (m[0] / m[1])
      else:
        NormDataframe[column] = (dataframe[column].astype(np.double) - m[0]) / m[1]
    elif column == 'reproduccion': 
      1==1
    else:
      NormDataframe[column] = (dataframe[column].astype(np.double) - m[0]) / m[1]
   
  return NormDataframe

In [ ]:
import pandas as pd, numpy as np
dataframe_part = pd.read_csv('drive/MyDrive/AI/evento_particular.csv')
dataframe_part = normalize(dataframe_part, meanStd)

In [ ]:
dataframe_part = dataframe_part.drop(['reproduccion'], axis=1)
dataframe_part = dataframe_part.drop(['fecha_reproduccion'], axis=1)
dataframe_part = dataframe_part.drop(['dia_semana'], axis=1)

In [ ]:
dataframe_part

In [ ]:


model.eval()
Norm_dataframe_part = normalize(dataframe_part, meanStd)

X_part = torch.from_numpy(Norm_dataframe_part.to_numpy()).type(torch.FloatTensor)

pred = model(X_part)

pred

#Invocación a predicciones y resultados